In [3]:
%pip install facenet_pytorch

^C


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 11.5 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.6 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------------------------- 2.4/198.5 MB 11.2 MB/s eta 0:00:18
   - -------------------------------------- 5.0/198.5 MB 11.6 MB/s eta 0:00:17
   - -------------------------------------- 7.1/198.5 MB 11.2 MB/s eta 0:00:18
   - -------------------------------------- 9.7/198.5 MB 11.4 MB/s eta 0:00:17
   -- ---------------------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 5.29.4 which is incompatible.


In [50]:
from facenet_pytorch import MTCNN
from PIL import Image, ImageEnhance, ImageStat
import torch
from torchvision import transforms

# 얼굴 탐지기 초기화
mtcnn = MTCNN(keep_all=False, device='cpu')  # 단일 얼굴만

def auto_brightness(image, target_mean=250):
    """
    현재 이미지의 밝기 평균을 측정해서,
    target_mean(예: 130)에 맞게 밝기 비율을 조정해주는 함수.
    """
    stat = ImageStat.Stat(image)
    mean = stat.mean[0]  # 흑백 이미지일 때는 채널이 1개

    # 밝기 보정 비율 계산
    brightness_factor = target_mean / (mean + 1e-5)

    # 너무 과한 보정은 방지 (안정화 범위 지정)
    brightness_factor = max(0.7, brightness_factor)

    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness_factor)


image_path = 'happy.jpeg' # 확인할 이미지
original_image = image_path
image = Image.open(image_path).convert("RGB")

image = auto_brightness(image, target_mean=250)


# 얼굴 crop
face = mtcnn(image)  # 결과: torch.Tensor [3, H, W]

if face is not None:
    # ⬇️ 전처리: Grayscale + Resize(48x48)
    transform = transforms.Compose([
        # transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224))
    ])

    # 이미지 값 스케일링
    face = (face * 255).clamp(0, 255).byte()

    # Tensor → PIL 이미지로 변환 후 전처리
    face_pil = transforms.ToPILImage()(face)
    face_gray_resized = transform(face_pil)

    # 저장
    face_gray_resized.save("224x224.jpg")
    print("얼굴 저장 완료: 224x224.jpg")

else:
    print("얼굴을 찾을 수 없습니다.")

얼굴 저장 완료: 224x224.jpg


In [12]:
%pip install timm

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.30.1-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 11.2 MB/s eta 0:00:00
Using cached huggingface_hub-0.30.1-py3-none-any.whl (481 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Note: you may need to restart the kernel to use updated packages.


In [51]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from facenet_pytorch import MTCNN
from model import EmotionSwin  # Swin 모델 정의

# 감정 라벨 정의
emotion_labels = {
    0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy",
    4: "Neutral", 5: "Sad", 6: "Surprise",
}

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 로딩
model = EmotionSwin(num_classes=7).to(device)
model.load_state_dict(torch.load('emotion_swin_last.pth', map_location=device))
model.eval()

# 이미지 경로 (48x48 grayscale 이미지)
image_path = '224x224.jpg'

# 🔄 전처리: 1채널 → 3채널 복제 → Resize → Tensor → Normalize
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),         # 흑백 → RGB 채널 복제
    transforms.Resize((224, 224)),                       # Swin 입력 크기 맞춤
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)      # [-1, 1] 범위 정규화
])

# 이미지 불러오기 & 전처리
image = Image.open(image_path).convert('L')
image = transform(image).unsqueeze(0).to(device)  # [1, 3, 224, 224]

# 예측
with torch.no_grad():
    outputs = model(image)
    probs = F.softmax(outputs, dim=1)
    predicted = torch.argmax(probs, dim=1).item()
    confidence = probs[0][predicted].item()

# 출력
print(f"🧠 감정 예측 결과: {emotion_labels[predicted]} ({confidence * 100:.2f}%)")

🧠 감정 예측 결과: Happy (100.00%)


In [52]:
# 이제 MODNet 배경 교체 스크립트를 실행합니다.
# --input-path, --ckpt-path, --output-path는 상황에 맞게 조정하세요.
!python -m demo.image_matting.colab.inference --input-path {original_image} --bg-code {predicted} --ckpt-path pretrained/modnet.onnx


[Info] 감정 코드 Happy → 필터 색상 (255, 215, 0)
[Done] saved to output\happy_replaced.jpeg
